In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
df=spark.read.csv('salesrecord.csv',inferSchema=True,header=True)

In [3]:
df

DataFrame[Region: string, Country: string, Item Type: string, Sales Channel: string, Order Priority: string, Order Date: string, Order ID: int, Ship Date: string, Units Sold: int, Unit Price: double, Unit Cost: double, Total Revenue: double, Total Cost: double, Total Profit: double]

In [4]:
df.show()

+--------------------+----------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|         Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+----------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|  Sub-Saharan Africa|    South Africa|         Fruits|      Offline|             M| 7/27/2012|443368995| 7/28/2012|      1593|      9.33|     6.92|     14862.69|  11023.56|     3839.13|
|Middle East and N...|         Morocco|        Clothes|       Online|             M| 9/14/2013|667593514|10/19/2013|      4611|    109.28|    35.84|    503890.08| 165258.24|   338631.84|
|Australia and Oce...|Papua New Guinea|           Meat|      Offl

In [5]:
df.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item Type: string (nullable = true)
 |-- Sales Channel: string (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Order ID: integer (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Units Sold: integer (nullable = true)
 |-- Unit Price: double (nullable = true)
 |-- Unit Cost: double (nullable = true)
 |-- Total Revenue: double (nullable = true)
 |-- Total Cost: double (nullable = true)
 |-- Total Profit: double (nullable = true)



In [6]:
df.count()

500000

In [7]:
#stastical properties of a column
d=input("Enter the Column Name:")
df.describe(d).show()

Enter the Column Name:Total Profit
+-------+------------------+
|summary|      Total Profit|
+-------+------------------+
|  count|            500000|
|   mean|392479.96458844113|
| stddev|378751.68881151016|
|    min|              2.41|
|    max|         1738700.0|
+-------+------------------+



In [8]:
#using filter function to find the sales of particular Country
i=input("Enter the Country:")
df.filter(df.Country==i).show()

Enter the Country:South Africa
+------------------+------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|            Region|     Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+------------------+------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Sub-Saharan Africa|South Africa|         Fruits|      Offline|             M| 7/27/2012|443368995| 7/28/2012|      1593|      9.33|     6.92|     14862.69|  11023.56|     3839.13|
|Sub-Saharan Africa|South Africa|           Meat|      Offline|             M| 7/30/2013|458715951| 8/11/2013|      3098|    421.89|   364.69|   1307015.22|1129809.62|    177205.6|
|Sub-Saharan Africa|South Africa|        Clothes|      Offline| 

In [10]:
#generalizing the filter function to user given column name and value.
d=input("Enter Column Name:")
i=input("Enter Value:")
df.filter(df[d]==i).show()

Enter Column Name:Country
Enter Value:India
+------+-------+-------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Region|Country|    Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+------+-------+-------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|  Asia|  India|       Snacks|       Online|             L|10/10/2012|440306556|11/20/2012|      5349|    152.58|    97.44|    816150.42| 521206.56|   294943.86|
|  Asia|  India|Personal Care|      Offline|             H|  3/5/2010|829856431| 4/15/2010|      1814|     81.73|    56.67|    148258.22| 102799.38|    45458.84|
|  Asia|  India|         Meat|      Offline|             C| 2/18/2010|631333053| 3/24/2010|      4335|    421.89|   364.69|   1828893.15|1580931.1

In [11]:
#finding covariance and correlation between two columns
a=input("Enter C1:")
b=input("Enter C2:")
print("Covariance:",df.cov(a,b))
print("Correlation:",df.corr(a,b))

Enter C1:Unit Price
Enter C2:Unit Cost
Covariance: 37567.56828820225
Correlation: 0.9860397737591339


In [12]:
#crearing RDD for the csv file
data = sc.textFile("salesrecord.csv")
data = data.map(lambda line: line.split(","))
data

PythonRDD[52] at RDD at PythonRDD.scala:53

In [4]:
import os
import findspark
findspark.init()
import pyspark
from pyspark.sql.functions import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

def convert_to_df(path):
    df=spark.read.format("csv").option("header","true").option("inferSchema","true").load(path)
    return df
    
def data_cleaning_replace(df): 

    for i in df.dtypes:
        if i[1] == 'string':
            c= df.groupBy(i[0]).count().sort(desc("count")).distinct() #pyspark sql has sql like function, so the logic to find the most frequent value is to display count of every distinct value in the column in descending order, and the 1st value will be most frequent one n it should be not null 
        
            df=df.fillna({i[0]:c.where(col(i[0]).isNotNull()).collect()[0][0]})

        elif i[1] == 'float' or i[1] =='int' or i[1]=='double':
    
            df=df.fillna({i[0]:df.select(i[0]).agg(avg(i[0])).collect()[0][0]}) #this function finds mean of int and float columns and fills empty cells with it 
        
    return df


In [5]:
path= r'salesrecord.csv'

In [6]:
df =  convert_to_df(path)


In [7]:
df

DataFrame[Region: string, Country: string, Item Type: string, Sales Channel: string, Order Priority: string, Order Date: string, Order ID: int, Ship Date: string, Units Sold: int, Unit Price: double, Unit Cost: double, Total Revenue: double, Total Cost: double, Total Profit: double]

In [7]:
df = data_cleaning_replace(df)

In [8]:
df

DataFrame[Region: string, Country: string, Item Type: string, Sales Channel: string, Order Priority: string, Order Date: string, Order ID: int, Ship Date: string, Units Sold: int, Unit Price: double, Unit Cost: double, Total Revenue: double, Total Cost: double, Total Profit: double]

NameError: name 'df' is not defined